## Importações

In [ ]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import laspy
from shapely.geometry import box, Polygon, MultiPolygon, GeometryCollection


# from train_split import TrainSplit

## TranSplit

In [ ]:
import geopandas
from shapely.geometry import Point, Polygon


class TrainSplit:
    @staticmethod
    def _get_records_in_plot_location(
        lidar_data: geopandas.GeoDataFrame, plot_location: geopandas.GeoDataFrame
    ) -> any:
        # return plot_location.contains(lidar_data)
        print(plot_location.items())
        return lidar_data.assign(
            **{key: lidar_data.within(geom) for key, geom in plot_location.items()}
        )

# Split Geometry

In [ ]:
def katana(geometry, threshold, count=0):
    print("Geometry")
    print(type(geometry))
    print(geometry)

    """Split a Polygon into two parts across it's shortest dimension"""

    bounds = geometry.bounds

    width = bounds[2] - bounds[0]

    height = bounds[3] - bounds[1]

    print("width, height")

    print(width, height)

    if max(width, height) <= threshold or count == 250:

        # either the polygon is smaller than the threshold, or the maximum

        # number of recursions has been reached

        return [geometry]

    if height >= width:

        # split left to right

        a = box(bounds[0], bounds[1], bounds[2], bounds[1] + height / 2)

        b = box(bounds[0], bounds[1] + height / 2, bounds[2], bounds[3])

    else:

        # split top to bottom

        a = box(bounds[0], bounds[1], bounds[0] + width / 2, bounds[3])

        b = box(bounds[0] + width / 2, bounds[1], bounds[2], bounds[3])

    result = []

    for d in (
        a,
        b,
    ):

        c = geometry.intersection(d)

        if not isinstance(c, GeometryCollection):

            c = [c]

        for e in c:

            if isinstance(e, (Polygon, MultiPolygon)):

                result.extend(katana(e, threshold, count + 1))

    if count > 0:
        return result

    # convert multipart into singlepart

    final_result = []

    for g in result:

        if isinstance(g, MultiPolygon):

            final_result.extend(g)

        else:

            final_result.append(g)

    return final_result

## Carregamento dos dados

In [ ]:
las = laspy.read(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/DUC_A01_2017_LiDAR/DUC_A01_2017_LAS/DUCL0001C0004.las"
)

In [ ]:
lidar_dataset = pd.DataFrame(las.xyz, columns=["X", "Y", "Z"])

In [ ]:
del las

In [ ]:
geo_lidar_dataset = geopandas.GeoDataFrame(
    lidar_dataset,
    geometry=geopandas.points_from_xy(lidar_dataset.X, lidar_dataset.Y),
    crs="EPSG:32720",
)

In [ ]:
del lidar_dataset

In [ ]:
inventory_plot_location: geopandas.GeoDataFrame = geopandas.read_file(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/DUC_A01_2016_PLOTLOCATION/duc_a01_2016_plotlocation.shx"
)

In [ ]:
inventory_plot_location.head()

In [ ]:
inventory_plot_location.plot()

In [ ]:
# gdf = inventory_plot_location['geometry'].to_crs({'proj':'cea'})

In [ ]:
# (gdf.area / 10**6).sum()

In [ ]:
inventory = pd.read_csv(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/DUC_A01_2016_inventory.csv",
    encoding="ISO-8859-1",
)

In [ ]:
inventory.head()

In [ ]:
inventory["plot"].unique()

In [ ]:
inventory_plot_location

## Merge inventário com a região

In [ ]:
geo_inventory_dataset = geopandas.GeoDataFrame(
    inventory,
    geometry=geopandas.points_from_xy(
        inventory["UTM.Easting"], inventory["UTM.Northing"]
    ),
    crs="EPSG:32720",
)

In [ ]:
geo_inventory_dataset.head()

In [ ]:
inventory_point_in_poly = geopandas.sjoin(
    geo_inventory_dataset,
    inventory_plot_location,
    predicate="within",
    lsuffix="left",
    rsuffix="right",
)

In [ ]:
inventory_point_in_poly

In [ ]:
del geo_inventory_dataset

## Encontrando os dados LiDAR no Inventário

In [ ]:
lidar_point_in_poly = geopandas.sjoin(
    geo_lidar_dataset,
    inventory_plot_location,
    predicate="within",
    lsuffix="left",
    rsuffix="right",
)

In [ ]:
lidar_point_in_poly

In [ ]:
lidar_point_in_poly["plot_ID"].unique()

## Analisando dados do inventário à nível de plot

In [ ]:
inventory_point_in_poly.head()

In [ ]:
(inventory_point_in_poly.groupby("plot")["D.class"].describe())

In [ ]:
(inventory_point_in_poly.groupby("plot")["DBH"].describe())

# Splitando os dados do inventário

In [ ]:
inventory_plot_location

In [ ]:
inventory_splits = inventory_plot_location.groupby("plot_ID").apply(
    lambda dataframe: katana(dataframe.geometry.iloc[0], 50, 0)
)

In [ ]:
inventory_splits

In [ ]:
inventory_splits = inventory_splits.explode()

In [ ]:
inventory_plot_location = inventory_plot_location.join(inventory_splits.rename("splits"), on="plot_ID")

inventory_plot_location

In [ ]:
del inventory_splits